### https://github.com/pyvista/pyvista/discussions/2567

In [1]:
import vtk
import pyvista as pv
from ipywidgets import interact

pv.set_jupyter_backend('trame')

pl = pv.Plotter()
#c = pv.Cone(resolution=6)
c = vtk.vtkConeSource()
c_actor = pl.add_mesh(c)
pl.show(jupyter_backend='client')

def plot(resolution=6):
    #c.shallow_copy(pv.Cone(resolution=resolution))
    c.SetResolution(resolution)
    pl.render()

a = interact(plot, resolution = (3,20,1))

Widget(value="<iframe src='http://localhost:45487/index.html?ui=P_0x7ff3ee0bff10_0&reconnect=auto' style='widt…

interactive(children=(IntSlider(value=6, description='resolution', max=20, min=3), Output()), _dom_classes=('w…

In [2]:
import numpy as np
import xarray as xr
import pyvista as pv
import matplotlib

In [3]:
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox

In [4]:
ds = xr.open_dataset("https://thredds-su.ipsl.fr/thredds/dodsC/ipsl_thredds/brocksce/ICO/ICO.79.1jour.native.1_19790101_19790101_1D_inca_ges.nc")
ds

<xarray.Dataset>
Dimensions:               (cell: 16002, nvertex: 6, presnivs: 79,
                           time_counter: 24, axis_nbounds: 2)
Coordinates:
    lat                   (cell) float32 ...
    lon                   (cell) float32 ...
  * presnivs              (presnivs) float32 1.012e+05 1.009e+05 ... 5.297 1.483
    time_centered         (time_counter) datetime64[ns] ...
  * time_counter          (time_counter) datetime64[ns] 1979-01-01T00:30:00 ....
    time_instant          (time_counter) datetime64[ns] ...
Dimensions without coordinates: cell, nvertex, axis_nbounds
Data variables: (12/29)
    bounds_lon            (cell, nvertex) float32 ...
    bounds_lat            (cell, nvertex) float32 ...
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] ...
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] ...
    time_instant_bounds   (time_counter, axis_nbounds) datetime64[ns] ...
    pmid                  (time_counter, presnivs, cell) float64 ...
    ...                    ...
    vmrmcf                (time_counter, presnivs, cell) float64 ...
    vmrbe7                (time_counter, presnivs, cell) float64 ...
    vmrn2o                (time_counter, presnivs, cell) float64 ...
    vmrch4                (time_counter, presnivs, cell) float64 ...
    vmrco                 (time_counter, presnivs, cell) float64 ...
    vmrco2bih             (time_counter, presnivs, cell) float64 ...
Attributes:
    _NCProperties:                   version=1|netcdflibversion=4.6.0|hdf5lib...
    name:                            inca1d_ges
    description:                     Created by xios
    title:                           Created by xios
    Conventions:                     CF-1.6
    timeStamp:                       2022-Apr-05 08:59:15 GMT
    uuid:                            8982bdb7-b650-45c4-896c-e228bec0cb5a
    history:                         Fri Apr 15 12:05:37 2022: ncatted -O -a ...
    NCO:                             4.6.9
    DODS_EXTRA.Unlimited_Dimension:  time_counter

In [5]:
blon = ds['bounds_lon'].to_numpy()
blat = ds['bounds_lat'].to_numpy()
nvertex = blon.shape[-1]
print(nvertex)

blon = blon.reshape(-1, nvertex)
blat = blat.reshape(-1, nvertex)
print(blon.shape)

arr = ds['bounds_lon'].to_numpy()
blon = arr.reshape(-1, arr.shape[-1])
arr = ds['bounds_lat'].to_numpy()
blat = arr.reshape(-1, arr.shape[-1])

6
(16002, 6)


In [6]:
deg2rad = np.pi/180.
x = np.cos(blat*deg2rad)*np.cos(blon*deg2rad)
y = np.cos(blat*deg2rad)*np.sin(blon*deg2rad)
z = np.sin(blat*deg2rad)

In [7]:
points = np.stack((x,y,z), axis=2).reshape(x.size, 3)
faces = np.arange(x.shape[0] * nvertex).reshape(x.shape[0], nvertex)
faces = np.insert(faces, 0, nvertex, axis=1)

In [8]:
mesh = pv.PolyData(points, faces)
mesh

PolyData,Information
N Cells,16002
N Points,96012
N Strips,0
X Bounds,"-1.000e+00, 1.000e+00"
Y Bounds,"-9.998e-01, 9.998e-01"
Z Bounds,"-9.999e-01, 9.999e-01"
N Arrays,0


In [9]:
mesh = pv.PolyData(points, faces)
mesh.clean(inplace=True, tolerance=0.001)
mesh

PolyData,Information
N Cells,16002
N Points,32000
N Strips,0
X Bounds,"-1.000e+00, 1.000e+00"
Y Bounds,"-9.998e-01, 9.998e-01"
Z Bounds,"-9.999e-01, 9.999e-01"
N Arrays,0


In [10]:
var = ds['temp']
print(var.shape)

(24, 79, 16002)


In [11]:
kmax = var.shape[1]-1
lmax = var.shape[0]-1
kmax, lmax

(78, 23)

In [12]:
var = ds['temp'][0,0]
varmin = np.min(var)
varmax = np.max(var)
print("%f %f" %(varmin, varmax))

233.364045 302.943871


In [13]:
! wget -nc https://thredds-su.ipsl.fr/thredds/fileServer/ipsl_thredds/brocksce/pyvista/continents.vtk

Fichier «continents.vtk» déjà présent ; pas de récupération.



In [14]:
ds['ps'][0][0]

<xarray.DataArray 'ps' ()>
[1 values with dtype=float64]
Coordinates:
    lat            float32 ...
    lon            float32 ...
    time_centered  datetime64[ns] ...
    time_counter   datetime64[ns] 1979-01-01T00:30:00
    time_instant   datetime64[ns] ...
Attributes:
    long_name:           surface air pressure
    units:               Pa
    online_operation:    average
    interval_operation:  900 s
    interval_write:      1 h
    cell_methods:        time: mean (interval: 900 s)
    _ChunkSizes:         [    1 16002]

In [19]:
import pyvista as pv
from ipywidgets import interact

pv.set_jupyter_backend('trame')

pl = pv.Plotter()
pl.set_background('lightgray')

variable = 'temp'
kindex = 0
lindex = 0
var = ds[variable][lindex,kindex]
clim = [240, 400]
cmap = 'plasma'
show_edges = False

continents = pv.PolyData('continents.vtk')
pl.add_mesh(continents, color='white')
mesh_actor = pl.add_mesh(mesh, scalars=var, show_edges=show_edges, clim=clim, cmap=cmap, show_scalar_bar=False)
scalarBar_actor = pl.add_scalar_bar(title=variable, color='black')
pl.show(jupyter_backend='client')

def plot(clim, cmap, show_edges):
    global mesh_actor, scalarBar_actor
    pl.remove_actor(mesh_actor)
    pl.remove_actor(scalarBar_actor)
    mesh_actor = pl.add_mesh(mesh, scalars=var, show_edges=show_edges, clim=clim, cmap=cmap, show_scalar_bar=False)
    scalarBar_actor = pl.add_scalar_bar(title=variable, color='black')
    pl.render()

widget_edges = widgets.Checkbox(value=show_edges, description='Edges')
widget_k = widgets.IntSlider(min=0, max=kmax, step=1, description='k-index:')
widget_l = widgets.IntSlider(min=0, max=lmax, step=1, description='l-index:')
widget_clim = widgets.Button(description="Update range")
widget_cmap = widgets.Dropdown(options=matplotlib.colormaps(), value=cmap, description='cmap:')
widget_output = widgets.Output()

def on_edges_change(change):
    global show_edges
    show_edges = change['new']
    plot(clim, cmap, show_edges)

def on_kvalue_change(change):
    global var
    kindex = change['new']
    var = ds[variable][lindex, kindex]
    plot(clim, cmap, show_edges)

def on_lvalue_change(change):
    global var
    lindex = change['new']
    var = ds[variable][lindex, kindex]
    plot(clim, cmap, show_edges)

def on_cmap_change(change):
    global cmap
    cmap = change['new']
    plot(clim, cmap, show_edges)
    
def on_button_clicked(b):
    global clim
    varmin = float(np.min(var))
    varmax = float(np.max(var))
    clim = [varmin, varmax]
    plot(clim, cmap, show_edges)
    

widget_edges.observe(on_edges_change, names='value')
widget_k.observe(on_kvalue_change, names='value')
widget_l.observe(on_lvalue_change, names='value')
widget_cmap.observe(on_cmap_change, names='value')
widget_clim.on_click(on_button_clicked)

display( widgets.HBox([widgets.VBox([widget_cmap, widget_output]),
                       widgets.VBox([widget_k, widget_l]), 
                       widgets.VBox([widget_clim, widget_edges])]) )

Widget(value="<iframe src='http://localhost:45487/index.html?ui=P_0x7ff3aeeda230_0&reconnect=auto' style='widt…